In [1]:
!pip install --upgrade scikit-learn
!pip install --upgrade keras
!pip install --upgrade tensorflow
!pip install --upgrade pandas
!pip install --upgrade numpy
!pip install --upgrade matplotlib
!pip install --upgrade torch


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/b2/e4/30b53d839608d2212b97972a8516ba0c7e776ee1102eaa82624807b944cf/keras-3.0.1-py3-none-any.whl.metadata
  Using cached keras-3.0.1-py3-none-any.whl.metadata (4.8 kB)
Using cached keras-3.0.1-py3-none-any.whl (999 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for

Best practice to find the best parameters yet but too much time.


In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer


label_encoder = LabelEncoder()

# download the data from the given URLs and store them in dataframes
training_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/training_data.csv')
unlabelled_test_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the hyperparameters to search over
param_grid = {
    'hidden_dim1': [64, 128, 256],
    'hidden_dim2': [32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64]
}

# Function to train and evaluate a model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

for hidden_dim1 in param_grid['hidden_dim1']:
    for hidden_dim2 in param_grid['hidden_dim2']:
        for lr in param_grid['lr']:
            for batch_size in param_grid['batch_size']:
                # Create the model
                model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                                       hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
                criterion = nn.CrossEntropyLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Create DataLoaders
                train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
                test_loader = DataLoader(test_data, batch_size=batch_size)

                # Train and evaluate the model
                accuracy = train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer)

                # Update best parameters if current model is better
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

print(f"Best hyperparameters: {best_params}")

# Custom loop for hyperparameter tuning
best_accuracy = 0
best_params = {}

def train_and_evaluate_model_parallel(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

def hyperparameter_tuning(hidden_dim1, hidden_dim2, lr, batch_size):
    # Create the model
    model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                           hidden_dim2=hidden_dim2, output_dim=len(np.unique(y)))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Create DataLoaders
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    # Train and evaluate the model
    accuracy = train_and_evaluate_model_parallel(model, train_loader, test_loader, criterion, optimizer)

    return accuracy, {'hidden_dim1': hidden_dim1, 'hidden_dim2': hidden_dim2, 'lr': lr, 'batch_size': batch_size}

# Run hyperparameter tuning in parallel
results = Parallel(n_jobs=-1)(delayed(hyperparameter_tuning)(hidden_dim1, hidden_dim2, lr, batch_size) 
                              for hidden_dim1 in param_grid['hidden_dim1'] 
                              for hidden_dim2 in param_grid['hidden_dim2'] 
                              for lr in param_grid['lr'] 
                              for batch_size in param_grid['batch_size'])

# Find the best parameters
best_accuracy = 0
best_params = None
for accuracy, params in results:
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f"Best hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}%")

Best hyperparameters: {'hidden_dim1': 128, 'hidden_dim2': 128, 'lr': 0.001, 'batch_size': 64}


In [11]:
import torch
from torch.utils.data import DataLoader
from joblib import Parallel, delayed
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader
from joblib import Parallel, delayed
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import TensorDataset
import pandas as pd
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim

label_encoder = LabelEncoder()

# download the data from the given URLs and store them in dataframes
training_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/training_data.csv')
unlabelled_test_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/unlabelled_test_data.csv')

# Exemple de données
texts = training_data_df['sentence'].values.tolist()
labels = training_data_df['difficulty'].values.tolist()

# Création de la matrice TF-IDF
vectorizer = TfidfVectorizer(max_features=14585)  # Ajustez ce nombre pour qu'il corresponde à la taille de vos données
X = vectorizer.fit_transform(texts).toarray()
y = np.array(labels)
y = label_encoder.fit_transform(labels)  # 'labels' est votre liste ou tableau d'étiquettes

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Conversion en tenseurs PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

# Création de DataLoaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32  # Vous pouvez ajuster la taille du batch
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Neural Network Class
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Define the hyperparameters to search over
param_grid = {
    'hidden_dim1': [64, 128, 256],
    'hidden_dim2': [32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64]
}

# Function to train and evaluate a model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs=50):
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            output = model(inputs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
    


# Define the TextClassifier model
class TextClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x


# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).long()
y_test = torch.from_numpy(y_test).long()

# Define the hyperparameters
hidden_dim1 = 64
hidden_dim2 = 128
lr = 0.01
batch_size = 64

# Create the model
model = TextClassifier(input_dim=X_train.shape[1], hidden_dim1=hidden_dim1, 
                        hidden_dim2=hidden_dim2, output_dim=len(np.unique(y_train)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Create DataLoaders
train_data = torch.utils.data.TensorDataset(X_train, y_train)
test_data = torch.utils.data.TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# Train the model
for epoch in range(50):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy}%")

Accuracy: 41.25%
